In [7]:
!pip install OpenNMT-py

     |████████████████████████████████| 216 kB 5.0 MB/s 
     |████████████████████████████████| 57 kB 3.7 MB/s 
     |████████████████████████████████| 73 kB 1.7 MB/s 
     |████████████████████████████████| 16.6 MB 321 kB/s 
     |████████████████████████████████| 1.2 MB 48.3 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


**Experiment with data ratios**

In [ ]:
#Just for testing purpose(for deleting old text files)

import os
os.remove("experiment_1/src_test.txt")
os.remove("experiment_1/src_train.txt")
os.remove("experiment_1/src_valid.txt")
os.remove("experiment_1/tgt_test.txt")
os.remove("experiment_1/tgt_train.txt")
os.remove("experiment_1/tgt_valid.txt")
os.remove("experiment_1/run/example.vocab.src")
os.remove("experiment_1/run/example.vocab.tgt")
os.remove("experiment_1/run/model_step_1000.pt")
os.remove("experiment_1/run/model_step_500.pt")



In [6]:
#Experimenting with data ratios (reg and irreg verbs)

import random

# read in  data
with open("english_merged_copy.txt",'r') as file:
    lines = file.readlines()
        
with open("reg.txt",'w') as file:
    for line in lines:
        # find() returns -1 if no match is found
        if line.find("irreg") != -1:
            pass
        else:
            file.write(line)

with open("irreg.txt",'w') as file:
  for line in lines:
      if "irreg" in line:
        file.write(line)

with open("reg.txt",'r') as file:
    lines_reg = file.readlines()

with open("merged.txt",'w') as file:
    for line in lines:
        if line.find("irreg") != -1:
            file.write(line)
    for i in range(504):
        line_reg = random.choice(lines_reg)
        file.write(line_reg)

In [8]:
#create new set of src,trg,valid

# -*- coding: utf-8 -*-

import codecs
import _pickle as cPickle
import random
from collections import defaultdict
import pandas as pd

#set up output file
fout_src_train = codecs.open('experiment_1/src_train.txt','wb','utf-8')
fout_tgt_train = codecs.open('experiment_1/tgt_train.txt','wb','utf-8')
fout_src_valid = codecs.open('experiment_1/src_valid.txt','wb','utf-8')
fout_tgt_valid = codecs.open('experiment_1/tgt_valid.txt','wb','utf-8')
fout_src_test = codecs.open('experiment_1/src_test.txt','wb','utf-8')
fout_tgt_test = codecs.open('experiment_1/tgt_test.txt','wb','utf-8')

#read in  data
fin = codecs.open('merged.txt','rb','utf-8')

sources = []
targets = []

for line in fin:
	parts = line.strip().split()
	lemma = parts[2]
	form = parts[3]
	sources.append(' '.join(lemma))
	targets.append(' '.join(form))
fin.close()

pairs = list(zip(sources,targets))
random.shuffle(pairs)

#split into train and test
train = pairs[:int(.8*len(pairs))]
valid = pairs[int(.8*len(pairs)):int(.9*len(pairs))]
test = pairs[int(.9*len(pairs)):]

#write the outputs
for s,t in train:
	fout_src_train.write(s + '\n')
	fout_tgt_train.write(t + '\n')

for s,t in valid:
	fout_src_valid.write(s + '\n')
	fout_tgt_valid.write(t + '\n')

for s,t in test:
	fout_src_test.write(s + '\n')
	fout_tgt_test.write(t + '\n')



fout_src_train.close()
fout_tgt_train.close()
fout_src_valid.close()
fout_tgt_valid.close()
fout_src_test.close()
fout_tgt_test.close()


In [ ]:
!onmt_build_vocab -config config-exp1.yaml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-27 16:03:06,351 INFO] Counter vocab from 10000 samples.
[2022-03-27 16:03:06,351 INFO] Build vocab on 10000 transformed examples/corpus.
[2022-03-27 16:03:06,364 INFO] corpus_1's transforms: TransformPipe()
[2022-03-27 16:03:06,467 INFO] Counters src:38
[2022-03-27 16:03:06,467 INFO] Counters tgt:38


In [ ]:
!onmt_train -config config-exp1.yaml

[2022-03-27 16:05:56,790 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-03-27 16:05:56,790 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-27 16:05:56,790 INFO] Missing transforms field for valid data, set to default: [].
[2022-03-27 16:05:56,791 INFO] Parsed 2 corpora from -data.
[2022-03-27 16:05:56,791 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-03-27 16:05:56,791 INFO] Loading vocab from text file...
[2022-03-27 16:05:56,791 INFO] Loading src vocabulary from experiment_1/run/example.vocab.src
[2022-03-27 16:05:56,791 INFO] Loaded src vocab has 38 tokens.
[2022-03-27 16:05:56,791 INFO] Loading tgt vocabulary from experiment_1/run/example.vocab.tgt
[2022-03-27 16:05:56,792 INFO] Loaded tgt vocab has 38 tokens.
[2022-03-27 16:05:56,792 INFO] Building fields with vocab in counters...
[2022-03-27 16:05:56,792 INFO]  * tgt vocab size: 42.
[2022-03-27 16:05:56,792 INFO]  * src vocab size

In [ ]:
!onmt_translate -model experiment_1/run/model_step_1000.pt -src experiment_1/src_test.txt -output experiment_1/test-decoded.txt -gpu 0 -verbose -beam_size 12

[2022-03-27 16:05:46,308 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-03-27 16:05:46,403 INFO] 
SENT 1: ['@', 's', 'a', 'I', 'n']
PRED 1: @ s a I n d
PRED SCORE: -0.0884

[2022-03-27 16:05:46,403 INFO] 
SENT 2: ['d', 'I', 'g']
PRED 2: d I g d
PRED SCORE: -0.3997

[2022-03-27 16:05:46,403 INFO] 
SENT 3: ['r', 'e', 'I', 'p']
PRED 3: r e I p t
PRED SCORE: -0.0596

[2022-03-27 16:05:46,404 INFO] 
SENT 4: ['k', '&', 't', '@', 'l', 'O', 'g']
PRED 4: k & t @ l O l , d
PRED SCORE: -1.5656

[2022-03-27 16:05:46,404 

##Experiment_2:

In [ ]:
# -*- coding: utf-8 -*-
"""Match train/test split from experiment 1, by lemma."""
import codecs
import _pickle as cPickle
import random
from collections import defaultdict
import pandas as pd

#set up output file
fout_src_train = codecs.open('src_train_tagged.txt','wb','utf-8')
fout_tgt_train = codecs.open('tgt_train_tagged.txt','wb','utf-8')
fout_src_valid = codecs.open('src_valid_tagged.txt','wb','utf-8')
fout_src_test = codecs.open('src_test_tagged.txt','wb','utf-8')

#modify every line in the current valid data
fin = codecs.open('experiment_1/src_valid.txt','rb','utf-8')
for line in fin:
	fout_src_valid.write('<V;PST> ' + line)
fin.close()
#modify every line in the current test data
fin = codecs.open('experiment_1/src_test.txt','rb','utf-8')
for line in fin:
	fout_src_test.write('<V;PST> ' + line)
fin.close()

#read in a set of valid lemmas from the current train data
ok_lemmas = set()
fin =  codecs.open('experiment_1/src_train.txt','rb','utf-8')
for line in fin:
	ok_lemmas.add(line.strip())
fin.close()

#read in  data
fin = codecs.open('experiment_2/english_merged.txt','rb','utf-8')

sources = []
targets = []

for line in fin:
	parts = line.strip().split()
	lemma = parts[3]
	form = parts[4]
	vec = '<' + parts[2] + '> '
	if vec != 'V;NFIN' and ' '.join(lemma) in ok_lemmas:
		sources.append(vec + ' '.join(lemma))
		targets.append(' '.join(form))	
fin.close()

pairs = list(zip(sources,targets))
random.shuffle(pairs)

#split into train and test
train = pairs

#write the outputs
for s,t in train:
	fout_src_train.write(s + '\n')
	fout_tgt_train.write(t + '\n')

fout_src_train.close()
fout_tgt_train.close()
fout_src_valid.close()
fout_src_test.close()



In [ ]:
!onmt_build_vocab -config config-exp2.yaml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
path_tgt is None, it should be set unless the task is language modeling
path_tgt is None, it should be set unless the task is language modeling
Corpus test's weight should be given. We default it to 1 for you.
[2022-03-22 12:33:06,636 INFO] Counter vocab from 10000 samples.
[2022-03-22 12:33:06,636 INFO] Build vocab on 10000 transformed examples/corpus.
[2022-03-22 12:33:06,645 INFO] corpus_1's transforms: TransformPipe()
[2022-03-22 12:33:06,646 INFO] test's transforms: TransformPipe()
[2022-03-22 12:33:06,751 INFO] Counters src:64
[2022-03-22 12:33:06,751 INFO] Counters tgt:39


In [ ]:
!onmt_train -config config-exp2.yaml

[2022-03-22 12:33:13,043 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-03-22 12:33:13,043 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-03-22 12:33:13,044 INFO] Missing transforms field for valid data, set to default: [].
[2022-03-22 12:33:13,044 WARNING] path_tgt is None, it should be set unless the task is language modeling
[2022-03-22 12:33:13,044 INFO] Missing transforms field for test data, set to default: [].
[2022-03-22 12:33:13,044 WARNING] path_tgt is None, it should be set unless the task is language modeling
[2022-03-22 12:33:13,044 WARNING] Corpus test's weight should be given. We default it to 1 for you.
[2022-03-22 12:33:13,044 INFO] Parsed 3 corpora from -data.
[2022-03-22 12:33:13,044 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2022-03-22 12:33:13,044 INFO] Loading vocab from text file...
[2022-03-22 12:33:13,044 INFO] Loading src vocabulary from experiment_2/run/example.vo

In [ ]:
!onmt_translate -model experiment_2/run/model_step_1000.pt -src experiment_2/src_test_tagged.txt -output experiment_2/test-decoded.txt -gpu 0 -verbose -beam_size 12

[2022-03-22 12:34:02,774 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:282: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self._batch_index = self.topk_ids // vocab_size
[2022-03-22 12:34:02,827 INFO] 
SENT 1: ['<V;PST>', 'k', 'l', 'O', 'z', 'I', 't']
PRED 1: k l O z I t
PRED SCORE: -1.9059

[2022-03-22 12:34:02,827 INFO] 
SENT 2: ['<V;PST>', 'p', '@', 'U', 's', 't']
PRED 2: r r e g
PRED SCORE: -0.1182

[2022-03-22 12:34:02,827 INFO] 
SENT 3: ['<V;PST>', 'v', '&', 'l', 'j', 'U']
PRED 3: O d l u :
PRED SCORE: -4.2385

[2022-03-22 12:34:02,827 INFO] 
SENT 4: ['<V;PST>', 'I', 'N', 'k', 'A', ':', 's', '@', 'r', 'e'

In [ ]:
!onmt_translate -model experiment_2/run/model_step_1000.pt -src experiment_1_wugs/CELEXmod.txt -output experiment_1_wugs/output1.txt -gpu 0 -verbose -beam_size 12

Streaming output truncated to the last 5000 lines.
SENT 3255: ['m«lÈYn', 'm«lÈYnd', '26', 'malign', 'maligned', 'reg']
PRED 3255: r r e g
PRED SCORE: -0.0798

[2022-03-22 12:34:46,151 INFO] 
SENT 3256: ['m«rÈun', 'm«rÈund', '26', 'maroon', 'marooned', 'reg']
PRED 3256: r r e g
PRED SCORE: -0.0798

[2022-03-22 12:34:46,152 INFO] 
SENT 3257: ['mÈartÕ', 'mÈartÕd', '26', 'martyr', 'martyred', 'reg']
PRED 3257: r r e g
PRED SCORE: -0.0798

[2022-03-22 12:34:46,152 INFO] 
SENT 3258: ['mÈEn«s', 'mÈEn«st', '26', 'menace', 'menaced', 'reg']
PRED 3258: r r e g
PRED SCORE: -0.0798

[2022-03-22 12:34:46,152 INFO] 
SENT 3259: ['mÈEnstruet', 'mÈEnstruet«d', '26', 'menstruate', 'menstruated', 'reg']
PRED 3259: r r e g
PRED SCORE: -0.0798

[2022-03-22 12:34:46,152 INFO] 
SENT 3260: ['mIsrÈid', 'mIsrÈEd', '26', 'misread', 'irreg']
PRED 3260: r r e g
PRED SCORE: -0.3676

[2022-03-22 12:34:46,152 INFO] 
SENT 3261: ['mÈÃSrum', 'mÈÃSrumd', '26', 'mushroom', 'mushroomed', 'reg']
PRED 3261: r r e g
PRED SCOR